# 描述
基于pytorch实现Transformer模型    
代码参考：    
- [introduction-to-transformers kaggle](https://www.kaggle.com/code/alejopaullier/introduction-to-transformers)
- [Attention is All You Need论文复现代码 GitHub](https://github.com/tian-guo-guo/tian-guo-guo.github.io/blob/179a0646232128b39ea12156ff292b394e9f24a3/_posts/2019-09-15-6%20-%20Attention%20is%20All%20You%20Need%E8%AE%BA%E6%96%87%E5%A4%8D%E7%8E%B0%E4%BB%A3%E7%A0%81.md)

所需三方库版本：    
- python: 3.11.0
- torch: 2.2.0
- torchtext: 0.17.0
- spacy: 3.7.5
- numpy: 1.25.2

## Transformer模型结构
![Transformer Architecture](https://raw.githubusercontent.com/Andr-Robot/iMarkdownPhotos/refs/heads/master/Res/ml/transformer_architecture.png)

### 1. 导入必要的库

In [9]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

### 2. 多头注意力机制（Multi-Head Attention）

In [2]:
class MultiHeadAttention(nn.Module):
    def __init__(self, embed_size, heads):
        """
        MultiHeadAttention mechanism. 
        The input of the MultiHeadAttention mechanism is an embedding (or sequence of embeddings). 
        The embeddings are split into different parts and each part is fed into a head.
        args:
            embed_size (int): the size of the embedding
            heads (int): the number of heads you wish to create
        """
        super(MultiHeadAttention, self).__init__()
        self.embed_size = embed_size # 512 in Transformer
        self.heads = heads # 8 in Transformer
        self.head_dim = embed_size // heads # 64 in Transformer

        assert (
            self.head_dim * heads == embed_size
        ), "Embedding size needs to be divisible by heads"
        
        # Embedding 会被投影到Query, Key and Value
        # 先投影到embed_size, 然后再分成N个维度为head_dim的部分
        self.values = nn.Linear(embed_size, embed_size)
        self.keys = nn.Linear(embed_size, embed_size)
        self.queries = nn.Linear(embed_size, embed_size)
        self.fc_out = nn.Linear(embed_size, embed_size)

    def forward(self, values, keys, query, mask):
        # 定义前向传播方法，接收value, key, query和mask作为输入
        # Values, Keys and Queries have size: (batch_size, sequence_len, embedding_size)
        batch_size = query.shape[0]# Get number of training examples/batch size.
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]
        # === Pass through Linear Layer ===
        """
        step1: 线性变换
        对 Q、K、V 进行线性变换，将其投影到多个头的维度
        变换后的形状为：(batch_size, sequence_len, embed_size) -> (batch_size, sequence_len, heads, head_dim)
        """
        values = self.values(values)  # (batch_size, value_len, embed_size)
        keys = self.keys(keys)  # (batch_size, key_len, embed_size)
        queries = self.queries(query)  # (batch_size, query_len, embed_size)

        # Split the embedding into self.heads different pieces
        values = values.reshape(batch_size, value_len, self.heads, self.head_dim)
        keys = keys.reshape(batch_size, key_len, self.heads, self.head_dim)
        queries = queries.reshape(batch_size, query_len, self.heads, self.head_dim)

        # Einsum does matrix mult. for query*keys for each training example
        # with every other training example, don't be confused by einsum
        # it's just how I like doing matrix multiplication & bmm

        # einsum函数是NumPy和PyTorch中用于执行爱因斯坦求和约定的函数。它可以简洁地表示复杂的多维数组操作。
        """
        step2: 计算注意力得分
        使用 Einsum 函数(BMM)计算 Q 和 K 的点积，得到注意力分数
        """
        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
        # queries shape: (batch_size, query_len, heads, heads_dim),
        # keys shape: (batch_size, key_len, heads, heads_dim)
        # energy: (batch_size, heads, query_len, key_len)

        # Mask padded indices so their weights become 0
        # 如果提供了掩蔽矩阵，将其应用到得分上，将掩蔽位置处的得分设置为负无穷大
        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        # Normalize energy values similarly to seq2seq + attention
        # so that they sum to 1. Also divide by scaling factor for
        # better stability
        """
        step3: 计算注意力权重
        归一化注意力得分，使其和为1，并除以缩放因子以提高稳定性
        """
        attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3) 
        # attention shape: (batch_size, heads, query_len, key_len)
        # values shape: (batch_size, value_len, heads, heads_dim)
        # out after matrix multiply: (batch_size, query_len, heads, head_dim), then
        # we reshape and flatten the last two dimensions.
        """
        step4: 计算加权和
        使用注意力权重对V进行加权求和，并reshape结果
        变换后的形状为：(batch_size, query_len, heads, head_dim) -> (batch_size, query_len, embed_size)
        """
        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            batch_size, query_len, self.heads * self.head_dim
        )
        # Linear layer doesn't modify the shape, final shape will be
        # (batch_size, query_len, embed_size)
        """
        step5: 输出层
        通过最终的线性层输出结果
        """
        out = self.fc_out(out)
        return out

### 3. Transformer Layer
严格意义上应该算是Encoder Layer，因为Decoder Layer中会使用到这个部分，所以改成TransformerLayer，防止混淆   
1. Multi-Head Attention
2. Add & Norm
3. Feed Forward
4. Add & Norm again

![encoder_layer](https://raw.githubusercontent.com/Andr-Robot/iMarkdownPhotos/refs/heads/master/Res/ml/encoder_layer.png)

In [10]:
class TransformerLayer(nn.Module):
    """TransformerLayer 层的实现代码包含了多头注意力机制（Multi-Head Attention）和前馈神经网络（FeedForward Network, FFN）"""
    def __init__(self, embed_size, heads, dropout, forward_expansion=4):
        """
        TransformerLayer 层的实现代码包含了多头注意力机制（Multi-Head Attention）和前馈神经网络（FeedForward Network, FFN）
        args:
            embed_size (int): embedding的大小
            heads (int): 多头注意力机制的头数
            dropout (float): dropout的概率，用于防止过拟合
            forward_expansion (int): 前馈网络中隐藏层神经元的扩展倍数
        """
        super(TransformerLayer, self).__init__()
        # 用于计算注意力权重
        self.attention = MultiHeadAttention(embed_size, heads) 
        # Layer Normalization 层，用于稳定训练过程
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)
        # 前馈神经网络，由两个线性层和 ReLU 激活函数组成。
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size),
        )
        # Dropout 层，用于随机丢弃部分神经元，防止过拟合
        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        # 定义前向传播方法，接收value, key, query和mask作为输入
        # Values, Keys and Queries have size: (batch_size, query_len, embedding_size)
        attention = self.attention(value, key, query, mask) # attention shape: (batch_size, query_len, embedding_size)
        # Add skip connection, run through normalization and finally dropout
        # 将注意力输出与输入 query 相加（残差连接）
        x = self.dropout(self.norm1(attention + query)) # x shape: (batch_size, query_len, embedding_size)
        # 将归一化后的结果输入到前馈网络中
        forward = self.feed_forward(x) # forward shape: (batch_size, query_len, embedding_size)
        # 将前馈网络的输出与输入 x 相加（残差连接）
        out = self.dropout(self.norm2(forward + x)) # out shape: (batch_size, query_len, embedding_size)
        return out

### Encoder
1. Embedding
2. Positional Encoding
3. Transformer Block

![encoder](https://raw.githubusercontent.com/Andr-Robot/iMarkdownPhotos/refs/heads/master/Res/ml/encoder.png)

In [4]:
class Encoder(nn.Module):
    def __init__(self, src_vocab_size, embed_size, num_layers, heads,
        device, forward_expansion, dropout, max_length):
        """
        Transformer 编码器的初始化方法。

        Args:
            src_vocab_size (int): 源语言词汇表的大小。
            embed_size (int): 词嵌入的维度。
            num_layers (int): 编码器中 Transformer 层的数量。
            heads (int): 多头注意力机制的头数。
            device (str): 模型运行的设备，例如 "cuda" 或 "cpu"。
            forward_expansion (int): 前馈网络中隐藏层的扩展倍数。
            dropout (float): Dropout 概率。
            max_length (int): 输入序列的最大长度。
        """
        super(Encoder, self).__init__()
        self.embed_size = embed_size # size of the input embedding
        self.device = device # either "cuda" or "cpu"
        # Lookup table with an embedding for each word in the vocabulary
        # 词嵌入层：将输入的单词索引映射为高维向量
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size) 
        # Lookup table with a positional embedding for each word in the sequence
        # 位置嵌入层：为序列中的每个位置生成嵌入向量
        self.position_embedding = nn.Embedding(max_length, embed_size)
        # 多层 Transformer 层：堆叠 num_layers 个 TransformerLayer
        self.layers = nn.ModuleList(
            [
                TransformerLayer(
                    embed_size,
                    heads,
                    dropout=dropout,
                    forward_expansion=forward_expansion,
                )
                for _ in range(num_layers)
            ]
        )
        # Dropout 层：用于随机丢弃部分神经元，防止过拟合
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        """
        前向传播方法。

        args:
            x (torch.Tensor): 输入序列，形状为 (batch_size, sequence_length)，其中每个元素是单词的索引。
            mask (torch.Tensor): 用于屏蔽无效位置的掩码（例如填充位置）。

        return:
            out (torch.Tensor): 编码器的输出，形状为 (batch_size, sequence_length, embed_size)。
        """
        batch_size, seq_length = x.shape
        # positions is an arange from (0,seq_len), e.g: torch.tensor([[0,1,2,...,N], [0,1,2,...,N], ..., [0,1,2,...,N]])
        # 生成位置信息：创建一个从 0 到 seq_length-1 的位置索引
        # 例如，如果 seq_length=5，则 positions 为 [[0, 1, 2, 3, 4], [0, 1, 2, 3, 4], ...]
        positions = torch.arange(0, seq_length).expand(batch_size, seq_length).to(self.device)
        # 词嵌入 + 位置嵌入：将词嵌入和位置嵌入相加，得到最终的输入表示
        # 使用 Dropout 进行随机丢弃，防止过拟合
        x_x = self.word_embedding(x)
        p_p = self.position_embedding(positions)
        out = self.dropout((x_x + p_p))
        # In the Encoder the query, key, value are all the same, in the
        # decoder this will change. This might look a bit odd in this case.
        # 通过多层 Transformer 层进行编码
        for layer in self.layers:
            # 在编码器中，query、key 和 value 都是相同的（即 out）
            out = layer(out, out, out, mask)
        # output shape: torch.Size([batch_size, sequence_length, embedding_size])
        # 返回编码器的输出
        return out

### Decoder Layer
1. Masked Multi-Head Attention
2. Add & Norm
3. Multi-Head Attention
4. Add & Norm
5. Feed Forward
6. Add & Norm

![decoder layer](https://raw.githubusercontent.com/Andr-Robot/iMarkdownPhotos/refs/heads/master/Res/ml/decoder_layer.png)

In [5]:
class DecoderLayer(nn.Module):
    """Decoder 层的实现"""
    def __init__(self, embed_size, heads, forward_expansion, dropout, device):
        """
        Decoder 层的初始化方法。

        Args:
            embed_size (int): 输入嵌入的维度。
            heads (int): 多头注意力机制的头数。
            forward_expansion (int): 前馈网络中隐藏层的扩展倍数。
            dropout (float): Dropout 概率。
            device (str): 模型运行的设备，例如 "cuda" 或 "cpu"。
        """
        super(DecoderLayer, self).__init__()
        # Layer Normalization 层，用于稳定训练过程
        self.norm = nn.LayerNorm(embed_size)
        # 多头注意力机制，用于计算目标序列的自注意力
        self.attention = MultiHeadAttention(embed_size, heads=heads)
        # Transformer 层，用于结合编码器的输出和目标序列的表示
        self.transformer_block = TransformerLayer(
            embed_size, heads, dropout, forward_expansion
        )
        # Dropout 层，用于随机丢弃部分神经元，防止过拟合
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, value, key, src_mask, trg_mask):
        """
        前向传播方法

        Args:
            x (torch.Tensor): 目标序列的输入，形状为 (batch_size, target_sequence_len, embed_size)。
            value (torch.Tensor): 编码器提取的 value，形状为 (batch_size, source_sequence_len, embed_size)。
            key (torch.Tensor): 编码器提取的 key，形状为 (batch_size, source_sequence_len, embed_size)。
            src_mask (torch.Tensor): 源序列的掩码，用于屏蔽填充位置，形状为 (batch_size, 1, source_sequence_len)。
            trg_mask (torch.Tensor): 目标序列的掩码，用于屏蔽未来位置，形状为 (batch_size, target_sequence_len, target_sequence_len)。

        Returns:
            out (torch.Tensor): 解码器层的输出，形状为 (batch_size, target_sequence_len, embed_size)。
        """
        # 计算目标序列的自注意力
        attention = self.attention(x, x, x, trg_mask)
        # 残差连接 + Layer Normalization + Dropout
        query = self.dropout(self.norm(attention + x))
        # 通过 Transformer 层，结合编码器的输出和目标序列的表示
        out = self.transformer_block(value, key, query, src_mask)
        return out

### Decoder
1. Output Embedding
2. Decoder Block
3. Linear
4. Softmax

![decoder](https://raw.githubusercontent.com/Andr-Robot/iMarkdownPhotos/refs/heads/master/Res/ml/decoder.png)

In [6]:
class Decoder(nn.Module):
    """解码器的实现"""
    def __init__(self, trg_vocab_size, embed_size, num_layers, heads, forward_expansion,
        dropout, device, max_length):
        """
        Decoder 的初始化方法。

        Args:
            trg_vocab_size (int): 目标词汇表的大小。
            embed_size (int): 输入嵌入的维度。
            num_layers (int): 解码器中 DecoderLayer 的数量。
            heads (int): 多头注意力机制的头数。
            forward_expansion (int): 前馈网络中隐藏层的扩展倍数。
            dropout (float): Dropout 概率。
            device (str): 模型运行的设备，例如 "cuda" 或 "cpu"。
            max_length (int): 目标序列的最大长度。
        """
        super(Decoder, self).__init__()
        self.device = device
        #=== For each token in target vocab there is a token embedding ===
        # 词嵌入层：将目标序列的单词索引映射为高维向量# 
        self.word_embedding = nn.Embedding(trg_vocab_size, embed_size) 
        # 位置嵌入层：为序列中的每个位置生成嵌入向量
        self.position_embedding = nn.Embedding(max_length, embed_size)
        # 多层 DecoderLayer：堆叠 num_layers 个 DecoderLayer
        self.layers = nn.ModuleList(
            [
                DecoderLayer(embed_size, heads, forward_expansion, dropout, device)
                for _ in range(num_layers)
            ]
        )
        # 线性层：将解码器的输出映射为目标词汇表大小的概率分布
        self.fc_out = nn.Linear(embed_size, trg_vocab_size)
        # Dropout 层：随机丢弃部分神经元，防止过拟合
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_out, src_mask, trg_mask):
        """
        前向传播方法

        Args:
            x (torch.Tensor): 目标序列的输入，形状为 (batch_size, target_sequence_len)。
            enc_out (torch.Tensor): 编码器的输出，形状为 (batch_size, source_sequence_len, embed_size)。
            src_mask (torch.Tensor): 源序列的掩码，形状为 (batch_size, 1, source_sequence_len)。
            trg_mask (torch.Tensor): 目标序列的掩码，形状为 (batch_size, target_sequence_len, target_sequence_len)。

        Returns:
            out (torch.Tensor): 解码器的输出，形状为 (batch_size, target_sequence_len, trg_vocab_size)。
        """
        batch_size, seq_length = x.shape # x shape: (batch_size, target_sequence_len)
        # positions is an arange from (0,seq_len), e.g: torch.tensor([[0,1,2,...,N], [0,1,2,...,N], ..., [0,1,2,...,N]])
        # 生成位置信息：创建一个从 0 到 seq_length-1 的位置索引
        # 例如，如果 seq_length=5，则 positions 为 [[0,1,2,3,4], [0,1,2,3,4], ..., [0,1,2,3,4]]
        positions = torch.arange(0, seq_length).expand(batch_size, seq_length).to(self.device) # positions shape: (batch_size, target_sequence_len)
        # 词嵌入 + 位置嵌入：将词嵌入和位置嵌入相加，得到目标序列的初始表示
        # 使用 Dropout 进行随机丢弃
        x_x = self.word_embedding(x)
        p_p = self.position_embedding(positions)
        x = self.dropout((x_x + p_p))

        # 通过多层 DecoderLayer
        for layer in self.layers:
            # 在 DecoderLayer 中，目标序列的表示与编码器的输出结合
            x = layer(x, enc_out, enc_out, src_mask, trg_mask)
        # 通过线性层，将解码器的输出映射为目标词汇表大小的概率分布
        out = self.fc_out(x)
        return out

In [7]:
class Transformer(nn.Module):
    """Transformer 模型的实现"""
    def __init__(self, src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx, embed_size=512,
                 num_layers=6, forward_expansion=4, heads=8, dropout=0, device="cpu", max_length=100):
        """
        Transformer 的初始化方法。

        Args:
            src_vocab_size (int): 源词汇表的大小。
            trg_vocab_size (int): 目标词汇表的大小。
            src_pad_idx (int): 源序列中的填充索引。
            trg_pad_idx (int): 目标序列中的填充索引。
            embed_size (int): 输入嵌入的维度。
            num_layers (int): 编码器和解码器中 EncoderLayer 和 DecoderLayer 的数量。
            heads (int): 多头注意力机制中的头数。
            forward_expansion (int): 前馈网络中隐藏层的扩展倍数。
            dropout (float): Dropout 概率。
            device (str): 模型运行的设备，例如 "cuda" 或 "cpu"。
            max_length (int): 输入序列的最大长度。
        """
        super(Transformer, self).__init__()
        # === Encoder ===
        self.encoder = Encoder(src_vocab_size, embed_size, num_layers, heads, device, forward_expansion, dropout, max_length)
        # === Decoder ===
        self.decoder = Decoder(trg_vocab_size, embed_size, num_layers, heads, forward_expansion, dropout, device, max_length)
        # 填充索引
        # 填充索引（如 <PAD>）用于表示填充的位置。
        # 在词汇表中，填充索引通常被映射为一个特殊的 token（如 0 或 1）。
        self.src_pad_idx = src_pad_idx # 源序列的填充索引
        self.trg_pad_idx = trg_pad_idx # 目标序列的填充索引
        self.device = device

    def make_src_mask(self, src):
        """
        生成源序列的掩码，由于屏蔽填充位置（padding）

        Args:
            src (torch.Tensor): 源序列，形状为 (batch_size, src_sequence_len)。

        Returns:
            src_mask (torch.Tensor): 源序列的掩码，形状为 (batch_size, 1, 1, src_sequence_len)，其中填充位置为 False，有效位置为 True。
        """
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        # (N, 1, 1, src_len)
        return src_mask.to(self.device)

    def make_trg_mask(self, trg):
        """
        生成目标序列的掩码，用于屏蔽未来位置。

        Args:
            trg (torch.Tensor): 目标序列，形状为 (batch_size, trg_sequence_len)。

        Returns:
            trg_mask (torch.Tensor): 目标序列的掩码，形状为 (batch_size, 1, trg_sequence_len, trg_sequence_len)，其中未来位置为 0，当前及之前位置为 1。
        
        Example: for a target of shape (batch_size=1, target_size=4)
        tensor([[[[1., 0., 0., 0.],
                  [1., 1., 0., 0.],
                  [1., 1., 1., 0.],
                  [1., 1., 1., 1.]]]])
        """
        N, trg_len = trg.shape
        trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(
            N, 1, trg_len, trg_len
        )
        return trg_mask.to(self.device)

    def forward(self, src, trg):
        """
        前向传播方法。

        Args:
            src (torch.Tensor): 源序列，形状为 (batch_size, src_sequence_len)。
            trg (torch.Tensor): 目标序列，形状为 (batch_size, trg_sequence_len)。

        Returns:
            out (torch.Tensor): 模型的输出，形状为 (batch_size, trg_sequence_len, trg_vocab_size)。
        """
        # 生成源序列掩码
        src_mask = self.make_src_mask(src) # src_mask shape: (batch_size, 1, 1, src_sequence_len)
        # 生成目标序列掩码
        trg_mask = self.make_trg_mask(trg) # trg_mask shape: (batch_size, 1, trg_sequence_len, trg_sequence_len)
        # 编码器输出
        enc_src = self.encoder(src, src_mask) # enc_src shape: (batch_size, src_sequence_len, embed_size)
        # 解码器输出
        out = self.decoder(trg, enc_src, src_mask, trg_mask) # out shape: (batch_size, trg_sequence_len, trg_vocab_size)
        return out

## 简单测试代码

In [8]:
if __name__ == "__main__":
    """
    测试代码
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # 定义输入数据
    x = torch.tensor([[1, 5, 6, 4, 3, 9, 5, 2, 0], [1, 8, 7, 3, 4, 5, 6, 7, 2]]).to(device)
    trg = torch.tensor([[1, 7, 4, 3, 5, 9, 2, 0], [1, 5, 6, 2, 4, 7, 6, 2]]).to(device)
    # 定义填充索引
    src_pad_idx = 0 # index of the padding token in source vocabulary
    trg_pad_idx = 0 # index of the padding token in target vocabulary
    # 定义词汇表大小
    src_vocab_size = 10 # number of unique tokens in source vocabulary
    trg_vocab_size = 10 # number of unique tokens in target vocabulary
    
    print(f"Input shape: {x.shape}")
    print(f"Target shape: {trg.shape}")
    print(f"Device available: {device}")
    # 初始化 Transformer 模型
    model = Transformer(src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx, device=device).to(device)

    out = model(x, trg[:, :-1])
    print(f"Output shape: {out.shape}")
    print(f"Output: {out}")

Input shape: torch.Size([2, 9])
Target shape: torch.Size([2, 8])
Device available: cpu
Output shape: torch.Size([2, 7, 10])
Output: tensor([[[-0.0301,  0.0609, -0.5596, -0.5251, -0.4659,  0.5850, -0.4768,
           0.3702,  1.9181,  0.5932],
         [-1.1331, -0.2159, -0.3208, -0.9711,  0.2481, -0.4936, -0.5312,
           0.2812,  2.0300,  0.3126],
         [-0.2906,  0.8850, -0.3910, -1.1984,  1.0430,  0.4958, -0.0374,
           0.4621,  0.9843,  0.2634],
         [ 0.2161,  0.2121,  0.5236, -0.9407,  0.3107, -0.2067,  0.4018,
           1.5001,  0.2991,  1.1865],
         [-0.2761, -0.1771, -0.9052, -1.4448,  0.2574, -0.0782,  0.5652,
           1.0252,  0.6613,  1.4735],
         [-0.4923,  0.5697,  0.3478, -0.3036,  0.4055,  0.6242,  0.2012,
           1.1592,  1.2393,  1.0222],
         [-0.6083, -0.5369, -0.3365, -0.5693,  0.6065, -0.7188, -0.8129,
           0.8671,  0.2657,  1.2910]],

        [[ 0.1963,  0.2490, -0.5154, -0.6422, -0.3840,  0.5401, -0.4969,
           0.371

## 模型训练
使用PyTorch和torchtext库来加载和预处理WMT'14 English-German数据集，训练和评估一个简单的Transformer模型。

In [14]:
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import Multi30k
from torch.utils.data import DataLoader, Dataset

# 定义字段
SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'

token_transform = {}
vocab_transform = {}

# 使用 spacy 进行分词
token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='de_core_news_sm')
token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')

# 构建词汇表
def yield_tokens(data_iter, language):
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}
    for data_sample in data_iter:
        yield token_transform[language](data_sample[language_index[language]])

"""
这里会无法下载数据，可以手动下载数据集，放到 ~/.torchtext/cache/Multi30k/ 文件夹下
"""
# We need to modify the URLs for the dataset since the links to the original dataset are broken
# Refer to https://github.com/pytorch/text/issues/1756#issuecomment-1163664163 for more info
# multi30k.URL["train"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz"
# multi30k.URL["valid"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz"
# multi30k.URL["test"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/mmt16_task1_test.tar.gz"

# 指定本地数据集路径
local_data_path = '/Users/leo/.torchtext/cache/Multi30k'  # 替换为你的数据集路径

# 确保数据集文件存在
assert os.path.exists(local_data_path), "数据集路径不存在，请检查路径是否正确"

# 加载本地数据集
train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE), root=local_data_path)
valid_iter = Multi30k(split='valid', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE), root=local_data_path)

"""
https://github.com/pytorch/text/issues/2221
mmt16_task1_test.tar.gz torchtext有bug，会导致加载失败
UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 37: invalid start byte
解决方案：
将mmt16_task1_test.tar.gz解压，并将文件移动到~/.torchtext/cache/Multi30k/datasets/Multi30k/目录下
1. tar -zxvf mmt16_task1_test.tar.gz
2. mv test.* datasets/Multi30k/
3. cp mmt16_task1_test.tar.gz datasets/Multi30k/
"""

test_iter = Multi30k(split='test', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE), root=local_data_path)


vocab_transform[SRC_LANGUAGE] = build_vocab_from_iterator(yield_tokens(train_iter, SRC_LANGUAGE), min_freq=2, specials=["<unk>", "<pad>", "<bos>", "<eos>"])
vocab_transform[TGT_LANGUAGE] = build_vocab_from_iterator(yield_tokens(train_iter, TGT_LANGUAGE), min_freq=2, specials=["<unk>", "<pad>", "<bos>", "<eos>"])

for vocab in vocab_transform.values():
    vocab.set_default_index(vocab["<unk>"])

# 定义常量
SRC_PAD_IDX = vocab_transform[SRC_LANGUAGE]["<pad>"]
TGT_PAD_IDX = vocab_transform[TGT_LANGUAGE]["<pad>"]
BATCH_SIZE = 128
MAX_LENGTH = 128
# DROUPOUT = 0.3
EMBEDDING_SIZE = 256
NUM_LAYERS = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 数据处理函数
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(torch.tensor([vocab_transform[SRC_LANGUAGE]["<bos>"]] + vocab_transform[SRC_LANGUAGE](token_transform[SRC_LANGUAGE](src_sample)) + [vocab_transform[SRC_LANGUAGE]["<eos>"]], dtype=torch.long))
        tgt_batch.append(torch.tensor([vocab_transform[TGT_LANGUAGE]["<bos>"]] + vocab_transform[TGT_LANGUAGE](token_transform[TGT_LANGUAGE](tgt_sample)) + [vocab_transform[TGT_LANGUAGE]["<eos>"]], dtype=torch.long))
    src_batch = nn.utils.rnn.pad_sequence(src_batch, padding_value=SRC_PAD_IDX)
    tgt_batch = nn.utils.rnn.pad_sequence(tgt_batch, padding_value=TGT_PAD_IDX)
    return src_batch, tgt_batch

# 创建数据加载器
"""
Transformer 的输入格式：
    Transformer 模型的输入通常是一个形状为 (批次大小, 序列长度) 的张量。
    例如，src 的形状应为 (batch_size, src_seq_len)，trg 的形状应为 (batch_size, trg_seq_len)。
DataLoader 的输出格式：
    默认情况下，DataLoader 返回的批次数据形状是 (序列长度, 批次大小)。
    例如，src_batch 的形状为 (src_seq_len, batch_size)，trg_batch 的形状为 (trg_seq_len, batch_size)。
"""
train_dataloader = DataLoader(list(train_iter), batch_size=BATCH_SIZE, collate_fn=collate_fn)
valid_dataloader = DataLoader(list(valid_iter), batch_size=BATCH_SIZE, collate_fn=collate_fn)
test_dataloader = DataLoader(list(test_iter), batch_size=BATCH_SIZE, collate_fn=collate_fn)

# 初始化模型
INPUT_DIM = len(vocab_transform[SRC_LANGUAGE])
OUTPUT_DIM = len(vocab_transform[TGT_LANGUAGE])

model = Transformer(INPUT_DIM, OUTPUT_DIM, SRC_PAD_IDX, TGT_PAD_IDX, EMBEDDING_SIZE, NUM_LAYERS, max_length=MAX_LENGTH, dropout=DROUPOUT, device=device).to(device)

# 定义优化器和损失函数
optimizer = optim.Adam(model.parameters(), lr=0.00005, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss(ignore_index=TGT_PAD_IDX)

# 使用学习率调度器
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

# 训练模型
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0
    for i, (src, tgt) in enumerate(iterator):
        # 调整形状
        src = src.transpose(0, 1)  # (序列长度, 批次大小) -> (批次大小, 序列长度)
        tgt = tgt.transpose(0, 1)  # (序列长度, 批次大小) -> (批次大小, 序列长度)
        src, tgt = src.to(device), tgt.to(device)
        optimizer.zero_grad()
        output = model(src, tgt[:, :-1])
        output_dim = output.shape[-1]
        # output = [batch size, tgt sent len - 1, output dim]
        # tgt = [batch size, tgt sent len]
        output = output.contiguous().view(-1, output_dim)
        tgt = tgt[:,1:].contiguous().view(-1)
        # output = [batch size * tgt sent len - 1, output dim]
        # tgt = [batch size * tgt sent len - 1]
        loss = criterion(output, tgt)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(iterator)

# 测试模型
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, (src, tgt) in enumerate(iterator):
            # 调整形状
            src = src.transpose(0, 1)  # (序列长度, 批次大小) -> (批次大小, 序列长度)
            tgt = tgt.transpose(0, 1)  # (序列长度, 批次大小) -> (批次大小, 序列长度)
            src, tgt = src.to(device), tgt.to(device)
            output = model(src, tgt[:, :-1])
            output_dim = output.shape[-1]
            # output = [batch size, tgt sent len - 1, output dim]
            # tgt = [batch size, tgt sent len]
            output = output.contiguous().view(-1, output_dim)
            tgt = tgt[:,1:].contiguous().view(-1)
            # output = [batch size * tgt sent len - 1, output dim]
            # tgt = [batch size * tgt sent len - 1]
            loss = criterion(output, tgt)
            epoch_loss += loss.item()
    return epoch_loss / len(iterator)

# 训练和评估
N_EPOCHS = 10
CLIP = 1
best_valid_loss = float('inf')

# 创建保存模型的目录
model_dir = 'models'
os.makedirs(model_dir, exist_ok=True)
model_path = os.path.join(model_dir, 'transformer_model.pt')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    print(f"Epoch {epoch+1}")
    train_loss = train(model, train_dataloader, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_dataloader, criterion)
    
    end_time = time.time()
    epoch_mins, epoch_secs = divmod(end_time - start_time, 60)
    
    print(f'Epoch: {epoch+1:02} | Time: {int(epoch_mins)}m {int(epoch_secs)}s')
    print(f'\tTrain Loss: {train_loss:.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f}')
    # 使用学习率调度器
    scheduler.step(valid_loss)
    # 保存最好的模型
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), model_path)

# 加载最好的模型并进行测试
model.load_state_dict(torch.load(model_path))
test_loss = evaluate(model, test_dataloader, criterion)
print(f'Test Loss: {test_loss:.3f}')

Epoch 1
Epoch: 01 | Time: 1m 35s
	Train Loss: 6.655
	 Val. Loss: 5.483
Epoch 2
Epoch: 02 | Time: 1m 36s
	Train Loss: 5.313
	 Val. Loss: 4.942
Epoch 3
Epoch: 03 | Time: 1m 36s
	Train Loss: 5.009
	 Val. Loss: 4.752
Epoch 4
Epoch: 04 | Time: 1m 36s
	Train Loss: 4.813
	 Val. Loss: 4.561
Epoch 5
Epoch: 05 | Time: 1m 34s
	Train Loss: 4.646
	 Val. Loss: 4.400
Epoch 6
Epoch: 06 | Time: 1m 31s
	Train Loss: 4.512
	 Val. Loss: 4.277
Epoch 7
Epoch: 07 | Time: 1m 31s
	Train Loss: 4.406
	 Val. Loss: 4.186
Epoch 8
Epoch: 08 | Time: 1m 31s
	Train Loss: 4.328
	 Val. Loss: 4.125
Epoch 9
Epoch: 09 | Time: 1m 31s
	Train Loss: 4.266
	 Val. Loss: 4.082
Epoch 10
Epoch: 10 | Time: 1m 32s
	Train Loss: 4.211
	 Val. Loss: 4.028
Test Loss: 4.030
